In [ ]:
import numpy as np 
import pandas as pd 
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing

data = pd.read_csv("../input/data.csv")

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

In [ ]:
def test_it_RF(data_test):
    clf = RandomForestClassifier(n_jobs=-1, n_estimators=70, max_depth=7, random_state=2016)
    return cross_val_score(clf, data_test.drop('shot_made_flag', 1), data_test.shot_made_flag,
                           scoring='log_loss', cv=10)

In [ ]:
# define the sort & enumeration function
def sort_encode(df, field):
    ct = pd.crosstab(df.shot_made_flag, df[field]).apply(lambda x:x/x.sum(), axis=0)
    temp = list(zip(ct.values[1, :], ct.columns))
    temp.sort()
    new_map = {}
    for index, (acc, old_number) in enumerate(temp):
        new_map[old_number] = index
    new_field = field + '_sort_enumerated'
    df[new_field] = df[field].map(new_map)
    return new_field

In [ ]:
data['angle'] = 99
data.loc[~(data.loc_x==0), 'angle'] = \
np.arctan(data.loc[~(data.loc_x==0), 'loc_y'] / data.loc[~(data.loc_x==0), 'loc_x'])
data.loc[data.loc_x == 0, 'angle'] = np.pi / 2 

In [ ]:
angle_range = np.arange(-1.58,1.68,0.1)
binned = pd.cut(data.angle, angle_range) # 将loc_x区间化
binned_df = pd.DataFrame(binned)
binned_df.columns = ['angle_bin']
data = pd.concat([data, binned_df],axis=1)

In [ ]:
sort_encode(data, 'action_type')

In [ ]:
sort_encode(data, 'angle_bin')

In [ ]:
# Impute
mode = data.action_type_sort_enumerated.mode()[0]
data.action_type_sort_enumerated.fillna(mode, inplace=True)
mode1 = data.angle_bin_sort_enumerated.mode()[0]
data.angle_bin_sort_enumerated.fillna(mode1, inplace=True)

In [ ]:
feature = ['action_type_sort_enumerated', 'shot_distance', 'angle_bin_sort_enumerated', \
                                                  'shot_made_flag']

In [ ]:
data_cv = data.loc[~data.shot_made_flag.isnull(), feature]

In [ ]:
test_it_RF(data_cv).mean()

In [ ]:
[x for x in feature]

In [ ]:
feature1 = feature
feature2 = ['action_type_sort_enumerated','shot_distance','angle_bin_sort_enumerated','shot_id']
train = data.loc[~data.shot_made_flag.isnull(), feature1]
test = data.loc[data.shot_made_flag.isnull(), feature2]

In [ ]:
clf = RandomForestClassifier(n_jobs=-1, n_estimators=70, max_depth=7, random_state=2016)

# Train and predict
clf.fit(train.drop('shot_made_flag',axis=1), train.shot_made_flag)
predictions = clf.predict_proba(test.drop('shot_id',axis=1))

# convert to CSV
submission = pd.DataFrame({'shot_id': test.shot_id, 'shot_made_flag': predictions[:, 1]})
submission[['shot_id', 'shot_made_flag']].to_csv('submission_RF.csv', index=False)